<a href="https://colab.research.google.com/github/antoreep-jana/YouTube_Code_Repositories/blob/main/Weekly%20Sessions/Weekly_Session__14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Custom Model Training Loop

In [ ]:
import tensorflow as tf

In [ ]:
loss_tracker = tf.keras.metrics.Mean(name="loss")
mae_metric = tf.keras.metrics.MeanAbsoluteError(name="mae")


class CustomModel(tf.keras.Model):

  def train_step(self, data):

    x, y = data 

    with tf.GradientTape() as tape:
      y_pred = self(x, training = True)

      #loss = self.compiled_loss(y, y_pred, regularization_losses = self.losses)
      loss = tf.keras.losses.mean_squared_error(y, y_pred)

    trainable_vars = self.trainable_variables

    gradients = tape.gradient(loss, trainable_vars)

    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # Update metrics (includes the metric that tracks the loss)
    #self.compiled_metrics.update_state(y, y_pred)
    loss_tracker.update_state(loss)
    mae_metric.update_state(y, y_pred)

    # Return a dict mapping metric names to current value
    return {"loss" : loss_tracker.result(), "mae" : mae_metric.result()}
    #return {m.name: m.result() for m in self.metrics}

  @property
  def metrics(self):
    return [loss_tracker, mae_metric]


import numpy as np

# Construct and compile an instance of CustomModel
inputs = tf.keras.Input(shape=(32,))
outputs = tf.keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)



Epoch 1/3
32/32 [==============================] - 0s 1ms/step - loss: 0.5284 - mae: 0.6080
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2576 - mae: 0.4042
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2314 - mae: 0.3831


Method-2

In [ ]:
import tensorflow as tf

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


batch_size = 32

print(x_train.shape, y_train.shape) # (50000, 32, 32, 3) (50000, 1)
print(x_test.shape, y_test.shape)   # (10000, 32, 32, 3) (10000, 1)


# train set / data 
x_train = x_train.astype('float32') / 255

# validation set / data 
x_test = x_test.astype('float32') / 255


y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# validation set / target 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)


# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(batch_size)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [ ]:
base = tf.keras.applications.VGG16(include_top = False, input_shape = (32,32,3), weights = 'imagenet')

model = tf.keras.models.Sequential([
                                    base, 
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(256, activation= 'relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2

In [ ]:
# define optimizer & loss func
optimizer = tf.keras.optimizers.Adam()

loss_fn = tf.keras.losses.CategoricalCrossentropy()
val_loss_fn = tf.keras.losses.CategoricalCrossentropy()
## Adding metrics 

train_acc = tf.keras.metrics.CategoricalAccuracy()
val_acc = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
from tqdm import tqdm

In [ ]:
"""
Steps for custom training

1. Epochs - 1 for loop
2. Within epochs - batches - another for loop
3. Calculate gradients under tape
    a. pred logits from forward pass
    b. calc loss value
4. calculate gradient using tape.gradient() of training variables wrt loss 
5. Update the weight based on the gradient. Run one step of gradient descent. 

"""
epochs = 5

for epoch in range(epochs):
  print(f"Epoch {epoch}/{epochs}")
  for step , (x_batch_train, y_batch_train) in enumerate(tqdm(train_dataset)):

    """ Training Begins"""  
    with tf.GradientTape() as tape:
      preds = model(x_batch_train, training = True) 

      loss = loss_fn(y_batch_train, preds)

    grads= tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_acc.update_state(y_batch_train, preds)





  print(f"Training Loss : {loss}")

  print(f"Training Accuracy : {train_acc.result()}")

  train_acc.reset_states()


  """ Validation Begins"""

  for step, (x_batch_val, y_batch_val) in enumerate(tqdm(val_dataset)):

    preds = model(x_batch_val, training = False)

    loss = loss_fn(y_batch_val, preds)

    val_acc.update_state(y_batch_val , preds)

  

  print(f"Validaton Loss : {loss}")
  print(f'Validation Accuracy : {val_acc.result()}')

  val_acc.reset_states()


  





Epoch 0/5


100%|██████████| 1563/1563 [02:22<00:00, 10.98it/s]


Training Loss : 1.5089772939682007
Training Accuracy : 0.2332800030708313


100%|██████████| 313/313 [00:07<00:00, 41.54it/s]


Validaton Loss : 1.9351825714111328
Validation Accuracy : 0.30869999527931213
Epoch 1/5


...

Training Loss : 1.6734575033187866
Training Accuracy : 0.4395599961280823


100%|██████████| 313/313 [00:07<00:00, 42.27it/s]


Validaton Loss : 1.5182762145996094
Validation Accuracy : 0.4309999942779541
Epoch 3/5


 64%|██████▍   | 1008/1563 [01:09<00:38, 14.60it/s]


KeyboardInterrupt: ignored

In [ ]:
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:

    preds = model(x, training = True)

    loss = loss_fn(y, preds)

  grads = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  train_acc.update_state(y, preds)
  return loss

@tf.function
def val_step(x, y):
  preds_val = model(x, training = False)
  #loss_val = val_loss_fn(y, preds)
  #print("X Shape -> ", x.shape)
  #print("YShape ->", y.shape)
  #print("Preds Shape -> ", preds_val.shape)
  val_loss = loss_fn(y, preds_val)
  val_acc.update_state(y, preds_val)
  return val_loss
  #return loss_val

In [ ]:
epochs = 5


for epoch in range(epochs):

  print(f"Epoch {epoch}/{epochs}")
  for step, (x_train_batch, y_train_batch) in enumerate(tqdm(train_dataset)):
    train_loss = train_step(x_train_batch, y_train_batch)
  
  print(f"Training Accuracy -> {train_acc.result()}", f" Loss : {train_loss}")

  train_acc.reset_states()
  for step, (x_val_batch, y_val_batch) in enumerate(tqdm(val_dataset)):
    val_loss = val_step(x_val_batch, y_val_batch)

  print(f"Validation Accuracy -> {val_acc.result()}", f" Loss : {val_loss}")

  val_acc.reset_states()


Epoch 0/5


100%|██████████| 1563/1563 [01:27<00:00, 17.92it/s]


Training Accuracy -> 0.0967399999499321  Loss : 2.304940700531006


100%|██████████| 313/313 [00:04<00:00, 65.01it/s]


Validation Accuracy -> 0.09932126849889755  Loss : 2.2992143630981445
Epoch 1/5


100%|██████████| 1563/1563 [01:26<00:00, 18.26it/s]